In [1]:
import numpy as np
import uproot, awkward as ak
import matplotlib.pyplot as plt
from math import sin, isnan, isfinite

%matplotlib inline

def ak_np(x):
    """awkward -> flat numpy, finite only"""
    if isinstance(x, ak.Array):
        x = ak.flatten(x, axis=None)
        x = ak.to_numpy(x)
    x = np.asarray(x)
    return x[np.isfinite(x)]

def stats(name, diff, tol):
    if diff.size == 0:
        print(f"{name}: EMPTY")
        return
    print(f"{name}: N={diff.size:,}  mean={diff.mean():.3e}  std={diff.std(ddof=1):.3e}  "
          f"max|Δ|={np.max(np.abs(diff)):.3e}  p99.9|Δ|={np.quantile(np.abs(diff), 0.999):.3e}")
    frac = np.mean(np.abs(diff) > tol)
    print(f"  > tol={tol:g}: {frac*100:.4f}%")


In [ ]:
CKF_ROOT = "/data/jlai/iris-hep/res_pT/output_pt_10/tracksummary_ckf.root"

with uproot.open(CKF_ROOT) as f:
    t = f["tracksummary"]
    cols = [c for c in [
        "res_ePT_fit","res_ePTrel_fit",
        "eQOP_fit","eTHETA_fit",
        "t_pt","t_p","t_theta",
        "truthMatched","hasFittedParams",
    ] if c in t.keys()]
    arr = t.arrays(cols, library="ak")

# mask: matched truth + has fitted params + finite inputs
m = ak.Array(np.ones(len(arr[cols[0]]), dtype=bool))
if "truthMatched" in arr.fields:
    m = m & (arr["truthMatched"] == 1)
if "hasFittedParams" in arr.fields:
    m = m & (arr["hasFittedParams"] == True)

if "t_pt" in arr.fields:
    tpt = arr["t_pt"]
else:
    tpt = arr["t_p"] * np.sin(arr["t_theta"]) if ("t_p" in arr.fields and "t_theta" in arr.fields) else None

pt_rec = None
if ("eQOP_fit" in arr.fields) and ("eTHETA_fit" in arr.fields):
    p_rec  = abs(1.0 / arr["eQOP_fit"])
    pt_rec = p_rec * np.sin(arr["eTHETA_fit"])

if (tpt is not None) and (pt_rec is not None):
    finite_mask = m & np.isfinite(tpt) & np.isfinite(pt_rec)
else:
    finite_mask = ak.Array(np.zeros(len(m), dtype=bool))

res_pt_branch    = ak_np(arr["res_ePT_fit"][finite_mask])         if "res_ePT_fit"    in arr.fields else np.array([])
res_ptrel_branch = ak_np(arr["res_ePTrel_fit"][finite_mask])      if "res_ePTrel_fit" in arr.fields else np.array([])
tpt_np           = ak_np(tpt[finite_mask])                        if tpt is not None else np.array([])
ptrec_np         = ak_np(pt_rec[finite_mask])                     if pt_rec is not None else np.array([])

res_pt_derived    = ptrec_np - tpt_np
res_ptrel_derived = np.divide(res_pt_derived, tpt_np, out=np.full_like(res_pt_derived, np.nan), where=tpt_np!=0)

# diffs (branch - derived)
d_abs = res_pt_branch - res_pt_derived
d_rel = res_ptrel_branch - res_ptrel_derived

# internal consistency check: res_ePTrel_fit ?= res_ePT_fit / t_pt
res_ptrel_from_abs = np.divide(res_pt_branch, tpt_np, out=np.full_like(res_pt_branch, np.nan), where=tpt_np!=0)
d_rel_internal = res_ptrel_branch - res_ptrel_from_abs

# print quick stats
print(f"Entries used: {d_abs.size:,}")
stats("Δ absolute pT residual", d_abs, tol=1e-6)   # GeV-ish tolerance
stats("Δ relative pT residual", d_rel, tol=1e-6)   # dimensionless
stats("Δ (branch rel − (branch abs / t_pt))", d_rel_internal, tol=1e-8)


Entries used: 921
Δ absolute pT residual: N=921  mean=0.000e+00  std=0.000e+00  max|Δ|=0.000e+00  p99.9|Δ|=0.000e+00
  > tol=1e-06: 0.0000%
Δ relative pT residual: N=921  mean=0.000e+00  std=0.000e+00  max|Δ|=0.000e+00  p99.9|Δ|=0.000e+00
  > tol=1e-06: 0.0000%
Δ (branch rel − (branch abs / t_pt)): N=921  mean=0.000e+00  std=0.000e+00  max|Δ|=0.000e+00  p99.9|Δ|=0.000e+00
  > tol=1e-08: 0.0000%
